In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
!pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloaders.train import BalancedMelSampler
from dataloaders.val import MelSampler
from dataloaders.utils import csv_to_dict

from losses import NpairsLoss, MovingAverageBCE
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks
from models import NUM_FRAMES, Classifier, DeepMetricLearning

In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def get_lr(epoch, lr, min_lr=1e-5, max_lr=1e-3, epochs=30, warmup_prop=0.1):   
    if epoch <= epochs * warmup_prop:
        return min_lr + max_lr - (max_lr * (epochs * warmup_prop - epoch) / (epochs * warmup_prop))
    else:
        return min_lr + max_lr - (max_lr * (epoch - epochs * warmup_prop) / (epochs - epochs * warmup_prop))

In [9]:
def main(fold_idx, saved_path):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = csv_to_dict(train_data.iloc[train_index])
    fold_valid_dict = csv_to_dict(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=64,
        max_length=NUM_FRAMES,
        is_train=True,
        n_classes=24,
        use_cutmix=True,
        cache=True,
        n_classes_in_batch=8,
        shuffle_aug=False,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        cache=True,
        max_length=NUM_FRAMES,
        is_train=False,
        use_cutmix=False,
        shuffle_aug=balanced_train_data_loader.shuffle_aug,
    )

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=True,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=50,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    step_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    ma_bce = MovingAverageBCE(
        train_data.iloc[train_index],
        start_apply_step=20 * step_per_epoch,
        momentum=0.9,
        name="moving_average_loss",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        moving_average_bce=ma_bce,
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(True, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"   # simple
pretrained_folder = "../logs/2021-01-30/42/"  # mixstyle + iunterpolate + cbam
log_folder = "../logs/"

In [11]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    tf.keras.backend.clear_session()
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(fold_idx,  log_folder)
    
    if DEBUG:
        break

Logging results to ../logs/2021-02-04/3/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Training Model 

Epoch 1/100
Instructions for updating:
Use `tf.linalg.matmul` instead
15/15 - 7s - loss: 3.6073 - val_loss: 3.3376
Epoch 2/100
15/15 - 6s - loss: 3.7888 - val_loss: 3.2511
Epoch 3/100
15/15 - 6s - loss: 3.4221 - val_loss: 3.1673
Epoch 4/100
15/15 - 6s - loss: 3.2474 - val_loss: 3.0311
Epoch 5/100
15/15 - 6s - loss: 2.8940 - val_loss: 3.1196
Epoch 6/100
15/15 - 6s - loss: 3.9352 - val_loss: 2.8323
Epoch 7/100
15/15 - 6s - loss: 3.9534 - val_loss: 2.6566
Epoch 8/100
15/15 - 6s - loss: 3.7747 - val_loss: 2.5488
Epoch 9/100
15/15 - 6s - loss: 2.8364 - val_loss: 2.5944
Epoch 10/100
15/15 - 6s - loss: 3.8230 - val_loss: 2.5806
Epoch 11/100
15/15 - 6s - loss: 3.8368 - val_loss: 2.5605
Epoch 12/100
15/15 - 6s - loss: 3.7978 - val_loss: 2.4344
Epoch 13/100
15/15 - 6s - loss: 3.6105 - val_loss: 2.3657
Epoch 14/100
15/15 - 6s - loss: 2.3945 - val_los

Epoch 29/100
15/15 - 5s - loss: 3.6556 - val_loss: 2.0668
Epoch 30/100
15/15 - 6s - loss: 3.7940 - val_loss: 2.0146
Epoch 31/100
15/15 - 5s - loss: 2.3718 - val_loss: 2.0147
Epoch 32/100
15/15 - 6s - loss: 2.3114 - val_loss: 1.9922
Epoch 33/100
15/15 - 5s - loss: 2.2787 - val_loss: 2.0813
Epoch 34/100
15/15 - 5s - loss: 3.5707 - val_loss: 1.9969
Epoch 35/100
15/15 - 5s - loss: 2.3018 - val_loss: 2.0427
Epoch 36/100
15/15 - 5s - loss: 3.6743 - val_loss: 1.9943
Epoch 37/100
15/15 - 5s - loss: 3.6185 - val_loss: 2.0808
Epoch 38/100
15/15 - 6s - loss: 3.5662 - val_loss: 1.9934
Epoch 39/100
15/15 - 6s - loss: 2.3326 - val_loss: 1.9321
Epoch 40/100
15/15 - 5s - loss: 2.1752 - val_loss: 1.9979
Epoch 41/100
15/15 - 6s - loss: 2.5354 - val_loss: 2.0035
Epoch 42/100
15/15 - 6s - loss: 2.3027 - val_loss: 1.9532
Epoch 43/100
15/15 - 5s - loss: 2.2939 - val_loss: 2.0471
Epoch 44/100
15/15 - 6s - loss: 2.2662 - val_loss: 1.9924
Epoch 45/100
15/15 - 6s - loss: 2.2963 - val_loss: 1.9689
Epoch 46/100
1

Epoch 66/100
15/15 - 6s - loss: 2.1705 - val_loss: 2.0643
Epoch 67/100
15/15 - 5s - loss: 2.1758 - val_loss: 2.1019
Epoch 68/100
15/15 - 6s - loss: 2.2324 - val_loss: 2.0809
Epoch 69/100
15/15 - 6s - loss: 2.1322 - val_loss: 2.0907
Epoch 70/100
15/15 - 5s - loss: 3.5037 - val_loss: 2.1061
Epoch 71/100
15/15 - 5s - loss: 2.2443 - val_loss: 2.0743
Epoch 72/100
15/15 - 5s - loss: 3.5204 - val_loss: 2.0582
Epoch 73/100
15/15 - 6s - loss: 3.6089 - val_loss: 2.0572
Epoch 74/100
15/15 - 5s - loss: 3.5356 - val_loss: 2.0998
Epoch 75/100
15/15 - 5s - loss: 2.1192 - val_loss: 2.0931
Epoch 76/100
15/15 - 6s - loss: 2.1619 - val_loss: 2.0380
Epoch 77/100
15/15 - 5s - loss: 2.2604 - val_loss: 2.0499
Epoch 78/100
15/15 - 6s - loss: 2.1228 - val_loss: 2.0283
Epoch 79/100
15/15 - 5s - loss: 2.2346 - val_loss: 2.0834
Epoch 80/100
15/15 - 6s - loss: 3.5448 - val_loss: 2.0850

-------------   Fold 4 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Training Model 

Epoch 1/100
15/15 - 7s

Epoch 46/100
15/15 - 6s - loss: 2.2231 - val_loss: 1.7447
Epoch 47/100
15/15 - 5s - loss: 2.1362 - val_loss: 1.7785
Epoch 48/100
15/15 - 5s - loss: 3.6249 - val_loss: 1.7924
Epoch 49/100
15/15 - 5s - loss: 2.1342 - val_loss: 1.8756
Epoch 50/100
15/15 - 6s - loss: 2.2326 - val_loss: 1.7700
Epoch 51/100
15/15 - 6s - loss: 3.4157 - val_loss: 1.7442
Epoch 52/100
15/15 - 5s - loss: 3.4687 - val_loss: 1.7299
Epoch 53/100
15/15 - 6s - loss: 2.1624 - val_loss: 1.7658
Epoch 54/100
15/15 - 6s - loss: 2.1464 - val_loss: 1.7118
